In [7]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [2]:
data = pd.read_csv('insurance.csv', sep=',')

**APPLY COLLUMN TRANSFORMER TO ENCODE SOME STRING COLLUMNS**

In [6]:
col_trans = ColumnTransformer([
    ('hot_encode', OneHotEncoder(handle_unknown='ignore'), ('sex', 'smoker', 'region'))
])

**UTILISE PIPELINE**

In [8]:
pipe = Pipeline([('col_trans', col_trans), ('bintree', DecisionTreeRegressor())])

**APPLY GRID SERACH TO SELECT HYPERPARAMETERS**

In [18]:
params = dict(
    bintree_max_depth= np.linspace(5, 500, 496), 
    bintree_min_samples_leaf = np.linspace(2, 100, 99),
)
bintree_search = GridSearchCV(pipe, params, scoring='neg_mean_absolute_error', verbose=1, n_jobs=12)
bintree_search.fit(data.loc[:, ~data.columns.isin(['charges'])], data['charges'])

Fitting 5 folds for each of 49104 candidates, totalling 245520 fits


ValueError: Invalid parameter bintree_max_depth for estimator Pipeline(steps=[('col_trans',
                 ColumnTransformer(transformers=[('hot_encode',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ('sex', 'smoker',
                                                   'region'))])),
                ('bintree', DecisionTreeRegressor())]). Check the list of available parameters with `estimator.get_params().keys()`.

In [14]:
np.linspace(2, 100, 99)

array([  2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,
        13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,
        24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,
        35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,
        46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,
        57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,
        68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,
        79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,
        90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.])